In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import oracledb
import getpass
from scipy import stats
from sqlalchemy import create_engine, text

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
def getODBconn(user, hostname, db, pw = None):
    if pw == None:
        print("Enter password for " + user + "@" + db + ": ")
        pw = getpass.getpass()
    connection=oracledb.connect(user=user, password=pw, host=hostname, service_name=db)
    engine = create_engine('oracle+oracledb://', creator=lambda: connection)
    return engine

In [4]:
eng = getODBconn('VALESA10_SQL', 'AIX47', 'BCDWHPRD')

Enter password for VALESA10_SQL@BCDWHPRD: 
········


In [5]:
sql = text('''
SELECT ADDR_NUM, PARENT_ADDR_NUM, NAME_ALPHA, ADDRESS_LINE_1, 
ADDRESS_LINE_2, ADDRESS_LINE_3, ADDRESS_LINE_4, CITY, STATE_CD, 
STATE, ZIP, COUNTY, COUNTRY_CD, COUNTRY, LATITUDE, LONGITUDE
    FROM CUSTOMER 
    WHERE country_cd = 'US'
''')

sql2 = text('''SELECT 
Territory.Territory_name,
Territory.First_name rep_first_name,
Territory.Last_name rep_last_name,
Territory.Nick_name rep_nick_name,
Territory.Name_alpha rep_name_alpha,
Territory.Territory_CD,
Territory.Email_address TerritoryEmail
,Territory.region_code RegionCode
,Region.regional_director region_vp_full_name
,Region.first_name region_vp_first_name
,Region.last_name region_vp_last_name
,Region.nick_name region_vp_nick_name
,Region.descr_001 region_name
FROM EDW.TERRITORY Territory
    JOIN EDW.REGION Region
        ON Territory.REGION_CODE = Region.REGION_CD
''')

In [6]:
cnx = eng.connect()
qry = cnx.execute(sql)
sql_df = pd.DataFrame(qry.fetchall())
sql_df.columns = qry.keys()

In [7]:
cnx = eng.connect()
qry2 = cnx.execute(sql2)
sql_df2 = pd.DataFrame(qry2.fetchall())
sql_df2.columns = qry2.keys()

In [8]:
sql_df2

,territory_name,rep_first_name,rep_last_name,rep_nick_name,rep_name_alpha,territory_cd,territoryemail,regioncode,region_vp_full_name,region_vp_first_name,region_vp_last_name,region_vp_nick_name,region_name
0,OPEN,KURT,SCHMIDT,KURT,"SCHMIDT,KURT M",49,KURT.SCHMIDT@BATESVILLE.COM,A2,JOHN R WALL,JOHN,WALL,JOHN,OPEN
1,OPEN,JOHN,WALL,JOHN,"WALL,JOHN R",55,JOHN.WALL@BATESVILLE.COM,A2,JOHN R WALL,JOHN,WALL,JOHN,OPEN
2,OPEN,JOHN,WALL,JOHN,"WALL,JOHN R",55A,JOHN.WALL@BATESVILLE.COM,A2,JOHN R WALL,JOHN,WALL,JOHN,OPEN
3,OPEN,KURT,SCHMIDT,KURT,"SCHMIDT,KURT M",WJ,KURT.SCHMIDT@BATESVILLE.COM,A2,JOHN R WALL,JOHN,WALL,JOHN,OPEN
4,OPEN,KURT,SCHMIDT,KURT,"SCHMIDT,KURT M",WJC,KURT.SCHMIDT@BATESVILLE.COM,A2,JOHN R WALL,JOHN,WALL,JOHN,OPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,OPEN,BONNIE,JEFFRIES,BONNIE,"JEFFRIES,BONNIE J",Z1,BONNIE.JEFFRIES@BATESVILLE.COM,L2,JERRY L JUNE,JERRY,JUNE,JERRY,GM INSIDE SALES
905,OPEN,STUART,FORD,STUART,"FORD,STUART E",ZSA,STUART.FORD@BATESVILLE.COM,L2,JERRY L JUNE,JERRY,JUNE,JERRY,GM INSIDE SALES
906,"FORD,STUART E",STUART,FORD,STUART,"FORD,STUART E",TV,STUART.FORD@BATESVILLE.COM,L2,JERRY L JUNE,JERRY,JUNE,JERRY,GM INSIDE SALES
907,OPEN,STUART,FORD,STUART,"FORD,STUART E",ZS,STUART.FORD@BATESVILLE.COM,L2,JERRY L JUNE,JERRY,JUNE,JERRY,GM INSIDE SALES


In [9]:
sql_df['addr_num'] = sql_df['addr_num'].astype(str)

In [10]:
scm = pd.read_excel('../data/raw/2024-09-01 Standard Coverage Model - JR.xlsx', sheet_name= 'CVG Model');

In [11]:
scm['Ship To #'] = scm['Ship To #'].astype(str)
scm['Parent #'] = scm['Parent #'].astype(str) 

In [12]:
scm_us = scm[(scm['Country'] == 'US') & (scm['State'] != 'PR')]

In [13]:
scm_us.shape

(37135, 127)

In [14]:
joined_df = pd.merge(scm_us, sql_df[['addr_num', 'latitude', 'longitude']], how='left', left_on='Ship To #', right_on='addr_num')

In [15]:
joined_df.head(2)

,Ship To #,Ship To Name,Parent #,Parent Name,City,State,Country,Division,Region Code,Region Desc,Territory Cd,Rep Name,Status \n(C-Open;\n CO-Closed),Customer Group Code,Customer Group,Customer Segment,Program Sub-Group 2,Program Sub-Group 1,Customer Spectrum,Representative Type,NAR Code,NAR Manager,Std Industry Cd,Customer Type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,Cost Center / CSC,CSC Desc,Serv Lvl,Delivery Frequency,Delivery Service Priority,Buyer Type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,Line 1,Line 2,Line 3,Zip Code,addr_num,latitude,longitude
0,131453,02550 CAPITAL MERCHANDISE & SERVICES,131453,02550 CAPITAL MERCHANDISE & SERVICES,WHEAT RIDGE,CO,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,CO,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,1682.0,1682,1682,1682,,8014,CSC DENVER INVENTORY,M T W R F,5 x per week,0,Non-Buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8578 W 32ND AVE,,,80033,131453,39.7621,-105.092974
1,152738,01227 RIVERSIDE MEMORIAL CHAPEL,152738,01227 RIVERSIDE MEMORIAL CHAPEL,NEW YORK,NY,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,C,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,2500.0,2050,2050,1658,NTL,8023,CSC MOONACHIE INVENTORY,M T W R F S,6 x per week,1,Buyer,31.0,18612.94,2.0,29.0,2.0,2480.81,0.0,0.0,1455.48,0.0,20068.42,30.0,16962.19,0.0,30.0,0.0,0.0,0.0,0.0,556.22,0.0,17518.41,364.0,245913.22,10.0,354.0,9.0,9872.31,0.0,0.0,17521.9,0.0,263435.12,396.0,291530.26,11.0,385.0,4.0,2764.46,0.0,0.0,20180.91,0.0,311711.17,392.0,264617.14,12.0,380.0,11.0,12339.7,0.0,0.0,18634.34,0.0,283251.48,442.0,319454.21,11.0,431.0,5.0,3421.45,0.0,0.0,23617.91,0.0,343072.12,424.0,310234.

In [16]:
joined_df_valid = joined_df[joined_df['latitude'] != 0]

In [17]:
joined_df_valid['territory_cd_short'] = joined_df_valid['Territory Cd'].str.strip().str[:2]

C:\Users\valesa10\AppData\Local\Temp\ipykernel_4108\374168498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df_valid['territory_cd_short'] = joined_df_valid['Territory Cd'].str.strip().str[:2]


In [38]:
joined_df_valid.head(3)

,Ship To #,Ship To Name,Parent #,Parent Name,City,State,Country,Division,Region Code,Region Desc,Territory Cd,Rep Name,Status \n(C-Open;\n CO-Closed),Customer Group Code,Customer Group,Customer Segment,Program Sub-Group 2,Program Sub-Group 1,Customer Spectrum,Representative Type,NAR Code,NAR Manager,Std Industry Cd,Customer Type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,Cost Center / CSC,CSC Desc,Serv Lvl,Delivery Frequency,Delivery Service Priority,Buyer Type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,Line 1,Line 2,Line 3,Zip Code,addr_num,latitude,longitude,territory_cd_short
0,131453,02550 CAPITAL MERCHANDISE & SERVICES,131453,02550 CAPITAL MERCHANDISE & SERVICES,WHEAT RIDGE,CO,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,CO,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,1682.0,1682,1682,1682,,8014,CSC DENVER INVENTORY,M T W R F,5 x per week,0,Non-Buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8578 W 32ND AVE,,,80033,131453,39.7621,-105.092974,G1
1,152738,01227 RIVERSIDE MEMORIAL CHAPEL,152738,01227 RIVERSIDE MEMORIAL CHAPEL,NEW YORK,NY,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,C,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,2500.0,2050,2050,1658,NTL,8023,CSC MOONACHIE INVENTORY,M T W R F S,6 x per week,1,Buyer,31.0,18612.94,2.0,29.0,2.0,2480.81,0.0,0.0,1455.48,0.0,20068.42,30.0,16962.19,0.0,30.0,0.0,0.0,0.0,0.0,556.22,0.0,17518.41,364.0,245913.22,10.0,354.0,9.0,9872.31,0.0,0.0,17521.9,0.0,263435.12,396.0,291530.26,11.0,385.0,4.0,2764.46,0.0,0.0,20180.91,0.0,311711.17,392.0,264617.14,12.0,380.0,11.0,12339.70,0.0,0.0,18634.34,0.0,283251.48,442.0,319454.21,11.0,431.0,5.0,3421.45,0.0,0.0,23617.91,0.0,

In [19]:
joined_df_valid['Region Code'] = joined_df_valid['Region Code'].str.strip()

C:\Users\valesa10\AppData\Local\Temp\ipykernel_4108\234007715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df_valid['Region Code'] = joined_df_valid['Region Code'].str.strip()


In [83]:
# Function to join names
def join_names(row):
    if row['rep_last_name'] != ' ':  # Check if rep_last_name is not empty
        return f"{row['rep_last_name']}, {row['rep_first_name']}"
    return row['rep_first_name']

# Apply the function
sql_df2['rep_name_match'] = sql_df2.apply(join_names, axis=1)


In [21]:
joined_df_valid.to_csv('../data/processed/scm-10-24-geocoded-cleaned.csv')

In [84]:
test = pd.merge(joined_df_valid, sql_df2, how='left', left_on='Territory Cd', right_on='territory_cd')

In [85]:
test.head(30).tail(3)

,Ship To #,Ship To Name,Parent #,Parent Name,City,State,Country,Division,Region Code,Region Desc,Territory Cd,Rep Name,Status \n(C-Open;\n CO-Closed),Customer Group Code,Customer Group,Customer Segment,Program Sub-Group 2,Program Sub-Group 1,Customer Spectrum,Representative Type,NAR Code,NAR Manager,Std Industry Cd,Customer Type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,Cost Center / CSC,CSC Desc,Serv Lvl,Delivery Frequency,Delivery Service Priority,Buyer Type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,Line 1,Line 2,Line 3,Zip Code,addr_num,latitude,longitude,territory_cd_short,territory_name,rep_first_name,rep_last_name,rep_nick_name,rep_name_alpha,territory_cd,territoryemail,regioncode,region_vp_full_name,region_vp_first_name,region_vp_last_name,region_vp_nick_name,region_name,rep_name_match
27,130454,FLANNER AND BUCHANAN SPEEDWAY MORTUARY,130454,FLANNER AND BUCHANAN SPEEDWAY MORTUARY,INDIANAPOLIS,IN,US,A DIVISION,H4,BATESVILLE,EQ,OPEN,C,,NaN,AEP,,AM2,Independent,Acct Exec,,NaN,FH,FP,1580.0,630,630,629,GRO,8004,CSC BATESVILLE,M T W R F S,6 x per week,5,Buyer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1828.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1828.00,1.0,1810.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1810.5,1.0,1828.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1828.00,1.0,1810.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1810.5,1.0,1810.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1810.5,4.0,6976.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6976.0,2020 MONTCALM,,,46202,130454,39.7944938,-86.18014,EQ,OPEN,RYAN,MCKEE,RYAN,"MCKEE,RYAN C",EQ,RYAN.MCKEE@BATESVILLE.COM,H4,JOHN B SHOWALTER,JOHN,SHOWALTER,JOHN,BATESVILLE,"MCKEE, RYAN"
28,291117,"NEW MEXICO MORTUARY SERVICE, INC",291117,"NEW MEXICO MORTUARY SERVICE, INC",ALBUQUERQUE,NM,US,GROWTH,L1,GM LARGE VENDOR,K3,"BLUE, EVA",C,,NaN,LVV,,LE1,Lg Emerging,Inside Sales,,NaN,FH,FP,1093.0,618,618,618,ANG,8045,CSC ALBUQUERQUE INVENTORY,M T W R F,5 x per week,5,Non-Buyer,NaN,NaN,NaN,NaN,NaN,

In [24]:
rep_addresses_df = pd.read_csv('../data/processed/rep-addresses-geocoded-cleaned.csv', index_col=0)

In [86]:
rep_addresses_df['name_alpha'] = rep_addresses_df['last_name'].str.upper() + ', ' + rep_addresses_df['name'].str.upper()

In [97]:
rep_addresses_df

,division,region_code,region_name,territory,name,last_name,fax/cell,street_address,city,state,zip,email,country,comments,rd?,regionals_to_sort_separate,notes,full_address,lat,lon,formatted,district,geo_name,housenumber,street,suburb,postcode,city.1,county,state.1,state_code,confidence,confidence_city_level,confidence_street_level,name_alpha
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN,NaN,"1610 Jefferson St., Kerrville, TX",30.038996,-99.129007,"1610 Jefferson Street, Kerrville, TX 78028, Un...",NaN,NaN,1610,Jefferson Street,NaN,78028,Kerrville,Kerr County,Texas,TX,1.000000,1.000000,1.000000,"ABSHIER, MICAH"
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN,NaN,"3930 Bethlehem Rd., Boones Mill, VA",37.067470,-80.018910,"3930 Bethlehem, Franklin County, VA 24065, Uni...",NaN,Bethlehem,3930,Bethlehem,NaN,24065,NaN,Franklin County,Virginia,VA,0.000000,NaN,NaN,"ABSHIRE, TODD"
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\r\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN,NaN,"33475 Berns Shore Dr., Adel, IA",41.553946,-93.954977,"33475 Berns Shore Drive, Adel, IA 50003, Unite...",NaN,NaN,33475,Berns Shore Drive,NaN,50003,Adel,Dallas County,Iowa,IA,0.000000,NaN,NaN,"ADAMSON, JIM"
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN,NaN,"1218 Fairfield Estates Lane, Crownsville, MD",39.032044,-76.608929,"1218 Fairfield Estates Lane, Crownsville, MD 2...",NaN,NaN,1218,Fairfield Estates Lane,NaN,21032,Crownsville,Anne Arundel County,Maryland,MD,1.000000,1.000000,1.000000,"AGER, BRIAN"
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\r\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN,NaN,"5225 N. New England Ave., Chicago, IL",41.976360,-87.798655,"5225 North New England Avenue, Chicago, IL 606...",NaN,NaN,5225,North New England Avenue,Norwood Park,60656,Chicago,Cook County,Illinois,IL,1.000000,1.000000,1.000000,"ALDRIN, JASON"
5,A,B5,LA/Portland,X3,Ruby,Alvarez,808-382-3308,1865 Laniloa Place,Wahiawa,HI,96786,ruby.alvarez@batesville.com,NaN,NaN,NaN,NaN,NaN,"1865 Laniloa Place, Wahiawa, HI",21.502509,-158.002609,"1865 Laniloa Place, Wahiawa, HI 96786, United ...",NaN,NaN,1865,Laniloa Place,NaN,96786,Wahiawa,Honolulu County,Hawaii,HI,1.000000,1.000000,1.000000,"ALVAREZ, RUBY"
6,A,B5,LA/Portland,REG,Angela,Anderson,303.808.3471,22003 E Peakview Dr,Aurora,CO,80016,angela.anderson@batesville.com,NaN,NaN,y,NaN,NaN,"22003 E Peakview Dr, Aurora, CO",39.599930,-104.731850,"22003 East Peakview Drive, Aurora, CO 80016, U...",NaN,NaN,22003,East Peakview Drive,NaN,80016,Aurora,Arapahoe County,Colorado,CO,1.000000,1.000000,1.000000,"ANDERSON, ANGELA"
7,A,B5,LA/Portland,HK,Devan,Antram,480.369.2248,2799 W. Bridger Drive,San Tan Valley,AZ,85142,devan.antram@batesville.com,NaN,NaN,NaN,NaN,NaN,"2799 W. Bridger Drive, San Tan Valley, AZ",33.189766,-111.596395,"2799 West Bridger Drive, San Tan Valley, AZ 85...",NaN,NaN,2799,West Bridger Drive,NaN,85142,San Tan Valley,Pinal County,Arizona,AZ,1.000000,1.000000,1.000000,"ANTRAM, DEVAN"
8,A,H5,Dallas / Memphis,X5,Bryan,Arceneaux,* 225.769.0012\r\n225.921.0007,417 Hidden Lake Ct.,Baton Rouge,LA,70810,bryan.arceneaux@batesville.com,NaN,NaN,NaN,NaN,NaN,"417 Hidden Lake Ct., Baton Rouge, LA",30.362421,-91.103464,"417 Hidden Lake Court, Lakeside, LA 70810, Uni...",NaN,NaN,417,Hidden Lake Court,NaN,70810,Lakeside,East Baton Rouge Parish,Louisiana,LA,0.810000,0.810000,0.810000,"ARCENEAUX, BRYAN"
9,A,H5,Dallas / Memphis,WB,Aaron,Batlle,817-688-3218,102 Caraway Dr,Waco,TX,76712,aaron.batlle@batesville.com,NaN,NaN,NaN,NaN,NaN,"102 Caraway Dr, Waco, TX",31.487154,-97.289196,"102 Caraway Drive, Waco, TX 76712, United Stat...",NaN,NaN

In [88]:
test.columns.values

array(['Ship To #', 'Ship To Name', 'Parent #', 'Parent Name', 'City',
       'State', 'Country', 'Division', 'Region Code', 'Region Desc',
       'Territory Cd', 'Rep Name', 'Status \n(C-Open;\n CO-Closed)',
       'Customer Group Code', 'Customer Group', 'Customer Segment',
       'Program Sub-Group 2', 'Program Sub-Group 1', 'Customer Spectrum',
       'Representative Type', 'NAR Code', 'NAR Manager',
       'Std Industry Cd', 'Customer Type', 'Tot Dths', 'Burials',
       'Adj. Burials', 'Opportunity', 'Program', 'Cost Center / CSC',
       'CSC Desc', 'Serv Lvl', 'Delivery Frequency',
       'Delivery Service Priority', 'Buyer Type', 'Burial Units Prior Mo',
       'Net Burial $$$ Prior Mo', 'Metal Units Prior Mo',
       'Wood Units Prior Mo', 'Feature Units Prior Mo',
       'Net Feature $$$ Prior Mo', 'Constellation Units Prior Mo',
       'Net Constellation $$$ Prior Mo', 'Net Cremation $$$ Prior Mo',
       'Net BI $$$ Prior Mo', 'Net Total $$$ Prior Mo',
       'Burial Units

In [89]:
# Dictionary to map old names to new names
rename_dict = {
    'Ship To #': 'ship_to_id',
    'Ship To Name': 'ship_to_name',
    'Parent #': 'parent_id',
    'Parent Name': 'parent_name',
    'City': 'city',
    'State': 'state',
    'Country': 'country',
    'Division': 'division',
    'Region Code': 'region_code',
    'Region Desc': 'region_name',
    'Territory Cd': 'territory_code',
    'Rep Name': 'rep_name',
    'Status \n(C-Open;\n CO-Closed)': 'open_status',
    'Customer Group Code': 'customer_group_code',
    'Customer Group': 'customer_group',
    'Customer Segment': 'customer_segment',
#     'Program Sub-Group 2':
#     'Program Sub-Group 1':
    'Customer Spectrum': 'customer_spectrum',
    'Representative Type': 'rep_type',
#     'NAR Code':
#     'NAR Manager':
    'Std Industry Cd': 'std_industry_code',
    'Customer Type': 'customer_type',
#     'Tot Dths':
#     'Burials':
#     'Adj. Burials':
#     'Opportunity':
#     'Program':
    'Cost Center / CSC': 'csc_id',
    'CSC Desc': 'csc_name',
#     'Serv Lvl':
#     'Delivery Frequency':
#     'Delivery Service Priority':
    'Buyer Type': 'buyer_type',
#     'Burial Units Prior Mo':
#     'Net Burial $$$ Prior Mo':
#     'Metal Units Prior Mo':
#     'Wood Units Prior Mo':
#     'Feature Units Prior Mo':
#     'Net Feature $$$ Prior Mo':
#     'Constellation Units Prior Mo':
#     'Net Constellation $$$ Prior Mo':
#     'Net Cremation $$$ Prior Mo':
#     'Net BI $$$ Prior Mo':
#     'Net Total $$$ Prior Mo':
#     'Burial Units Prior Mo PY':
#     'Net Burial $$$ Prior Mo PY':
#     'Metal Units Prior Mo PY':
#     'Wood Units Prior Mo PY':
#     'Feature Units Prior Mo PY':
#     'Net Feature $$$ Prior Mo PY':
#     'Constellation Units Prior Mo PY':
#     'Net Constellation $$$ Prior Mo PY':
#     'Net Cremation $$$ Prior Mo PY':
#     'Net BI $$$ Prior Mo PY':
#     'Net Total $$$ Prior Mo PY':
#     'Burial Units YTD':
#     'Net Burial $$$ YTD':
#     'Metal Units YTD':
#     'Wood Units YTD':
#     'Feature Units YTD':
#     'Net Feature $$$ YTD':
#     'Constellation Units YTD':
#     'Net Constellation $$$ YTD':
#     'Net Cremation $$$ YTD':
#     'Net BI $$$ YTD':
#     'Net Total $$$ YTD':
#     'Burial Units PYTD':
#     'Net Burial $$$ PYTD':
#     'Metal Units PYTD':
#     'Wood Units PYTD':
#     'Feature Units PYTD':
#     'Net Feature $$$ PYTD':
#     'Constellation Units PYTD':
#     'Net Constellation $$$ PYTD':
#     'Net Cremation $$$ PYTD':
#     'Net BI $$$ PYTD':
#     'Net Total $$$ PYTD':
#     'Burial Units Rolling 12 mo':
#     'Net Burial $$$ Rolling 12 mo':
#     'Metal Units Rolling 12 mo':
#     'Wood Units Rolling 12 mo':
#     'Feature Units Rolling 12 mo':
#     'Net Feature $$$ Rolling 12 mo':
#     'Constellation Units Rolling 12 mo':
#     'Net Constellation $$$ Rolling 12 mo':
#     'Net Cremation $$$ Rolling 12 mo':
#     'Net BI $$$ Rolling 12 mo':
#     'Net Total $$$ Rolling 12 mo':
#     'Burial Units Rolling 12 mo PY':
#     'Net Burial $$$ Rolling 12 mo PY':
#     'Metal Units Rolling 12 mo PY':
#     'Wood Units Rolling 12 mo PY':
#     'Feature Units Rolling 12 mo PY':
#     'Net Feature $$$ Rolling 12 mo PY':
#     'Constellation Units Rolling 12 mo PY':
#     'Net Constellation $$$ Rolling 12 mo PY':
#     'Net Cremation $$$ Rolling 12 mo PY':
#     'Net BI $$$ Rolling 12 mo PY':
#     'Net Total $$$ Rolling 12 mo PY':
#     'Burial Units PY':
#     'Net Burial $$$ PY':
#     'Metal Units PY':
#     'Wood Units PY':
#     'Feature Units PY':
#     'Net Feature $$$ PY':
#     'Constellation Units PY':
#     'Net Constellation $$$ PY':
#     'Net Cremation $$$ PY':
#     'Net BI $$$ PY':
#     'Net Total $$$ PY':
#     'Burial Units 2PY':
#     'Net Burial $$$ 2PY':
#     'Metal Units 2PY':
#     'Wood Units 2PY':
#     'Feature Units 2PY':
#     'Net Feature $$$ 2PY':
#     'Constellation Units 2PY':
#     'Net Constellation $$$ 2PY':
#     'Net Cremation $$$ 2PY':
#     'Net BI $$$ 2PY':
#     'Net Total $$$ 2PY':
    'Line 1': 'address_line_1',
    'Line 2': 'address_line_2',
    'Line 3': 'address_line_3',
    'Zip Code': 'zip_code',
#     'addr_num':
    'latitude': 'lat',
    'longitude': 'lon',
#     'territory_cd_short':
#     'territory_name':
#     'rep_first_name':
#     'rep_last_name':
#     'rep_nick_name':
#     'rep_name_alpha':
#     'territory_cd':
#     'territoryemail':
#     'regioncode':
#     'region_vp_full_name':
#     'region_vp_first_name':
#     'region_vp_last_name':
#     'region_vp_nick_name':
    'region_name': 'region_name_2'
#     'rep_name_match':
}

In [90]:
# Rename columns
test.rename(columns=rename_dict, inplace=True)

In [92]:
test.head(80)

,ship_to_id,ship_to_name,parent_id,parent_name,city,state,country,division,region_code,region_name,territory_code,rep_name,open_status,customer_group_code,customer_group,customer_segment,Program Sub-Group 2,Program Sub-Group 1,customer_spectrum,rep_type,NAR Code,NAR Manager,std_industry_code,customer_type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,csc_id,csc_name,Serv Lvl,Delivery Frequency,Delivery Service Priority,buyer_type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,address_line_1,address_line_2,address_line_3,zip_code,addr_num,lat,lon,territory_cd_short,territory_name,rep_first_name,rep_last_name,rep_nick_name,rep_name_alpha,territory_cd,territoryemail,regioncode,region_vp_full_name,region_vp_first_name,region_vp_last_name,region_vp_nick_name,region_name_2,rep_name_match
0,131453,02550 CAPITAL MERCHANDISE & SERVICES,131453,02550 CAPITAL MERCHANDISE & SERVICES,WHEAT RIDGE,CO,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,CO,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,1682.0,1682,1682,1682,,8014,CSC DENVER INVENTORY,M T W R F,5 x per week,0,Non-Buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8578 W 32ND AVE,,,80033,131453,39.7621,-105.092974,G1,HOUSE ACCOUNTS,G1 HOUSE,,G1 HOUSE,G1 HOUSE ACCOUNTS,G1,,E1,,,,,HOUSE REGION,G1 HOUSE
1,152738,01227 RIVERSIDE MEMORIAL CHAPEL,152738,01227 RIVERSIDE MEMORIAL CHAPEL,NEW YORK,NY,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,C,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,2500.0,2050,2050,1658,NTL,8023,CSC MOONACHIE INVENTORY,M T W R F S,6 x per week,1,Buyer,31.0,18612.94,2.0,29.0,2.0,2480.81,0.0,0.00,1455.48,0.0,20068.42,30.0,16962.19,0.0,30.0,0.0,0.00,0.0,0.00,556

In [93]:
test = test.drop(['addr_num', 'territory_cd'], axis=1)

In [95]:
test[test['territory_name'] != test['rep_name']]

,ship_to_id,ship_to_name,parent_id,parent_name,city,state,country,division,region_code,region_name,territory_code,rep_name,open_status,customer_group_code,customer_group,customer_segment,Program Sub-Group 2,Program Sub-Group 1,customer_spectrum,rep_type,NAR Code,NAR Manager,std_industry_code,customer_type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,csc_id,csc_name,Serv Lvl,Delivery Frequency,Delivery Service Priority,buyer_type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,address_line_1,address_line_2,address_line_3,zip_code,lat,lon,territory_cd_short,territory_name,rep_first_name,rep_last_name,rep_nick_name,rep_name_alpha,territoryemail,regioncode,region_vp_full_name,region_vp_first_name,region_vp_last_name,region_vp_nick_name,region_name_2,rep_name_match
104,150578,"SCHOPPENHORST, UNDERWOOD & BROOKS F. H.",150578,"SCHOPPENHORST, UNDERWOOD & BROOKS F. H.",SHEPHERDSVILLE,KY,US,A DIVISION,H4,BATESVILLE,R4A,OPEN,C,,NaN,SEP,,URN,Strategic,Sr. Acct Exec,,NaN,FH,FP,505.0,375,375,368,PAA,8071,CSC LOUISVILLE INVENTORY,M T W R F S,6 x per week,5,Buyer,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,290.0,290.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,416.0,416.00,7.0,5749.40,6.0,1.0,0.0,0.00,7.0,5749.40,0.0,3344.0,9093.40,88.0,77240.66,73.0,15.0,8.0,12986.9,69.0,48801.85,0.00,2822.0,80062.66,7.0,5749.40,6.0,1.0,0.0,0.00,7.0,5749.40,0.00,3770.0,9519.40,105.0,88381.66,87.0,18.0,9.0,14877.9,85.0,58051.85,0.00,2960.0,91341.66,88.0,77240.66,73.0,15.0,8.0,12986.9,69.0,48801.85,0.00,3248.0,80488.66,333.0,284589.79,263.0,70.0,23.0,33922.1,263.0,178884.75,0.00,2744.0,287333.79,FUENRAL HOME LLC,4895 NORTH PRESTON,,40165-9223,38.009745,-85.723675,R4,"KERKER,BRANDON J",BRANDON,KERKER,BRANDON,"KERKER,BRANDON J",BRANDON.KERKER@BATESVILLE.COM,H4,JOHN B SHOWALTER,JOHN,SHOWALTER,JOHN,BATESVILLE,"KERKER, BRANDON"
119,146953,GROCE FUNERAL HOME,146953,GROCE FUNERAL HOME,ASHEVILLE,NC,US,A DIVISION,H2,CHARLOTTE,ZE,OPEN,C,,NaN,AEP,,,Independent,Acct Exec,,NaN,FH,FP,550.0,

In [96]:
# test.to_csv('../data/processed/scm-and-rvps-10-24-geocoded-cleaned.csv')

In [110]:
seg_df = pd.read_excel('../../c-nav/data/raw/CC16&17 Load Oct.24.xlsx', sheet_name= 'List');

In [120]:
seg_df['Customer Number'] = seg_df['Customer Number'].astype(str)

In [121]:
seg_df2 = seg_df[['Customer Number', 'Original', 'Customer Spectrum', 'CC23', 'CC28',
       'Sum of Total Deaths - Parent', 'Sum of Burials - Parent',
       'Adj Burials', 'Burial Units Rolling 12 mo - Parent',
       'Buying Program', 'Firm Size', 'Partner / Vendor', 'Retain / Grow',
       'Retain/Grow Code', 'Bucket', 'CC16', 'CC17', 'CC17.1']] 

In [122]:
seg_df2

,Customer Number,Original,Customer Spectrum,CC23,CC28,Sum of Total Deaths - Parent,Sum of Burials - Parent,Adj Burials,Burial Units Rolling 12 mo - Parent,Buying Program,Firm Size,Partner / Vendor,Retain / Grow,Retain/Grow Code,Bucket,CC16,CC17,CC17.1
0,127689,Lg Emerging,Lg Emerging,ANG,LE1,1240.0,591,591,4,NaN,Large,Vendor,Grow,G,Lg Emerging Large Vendor,LLV,G,NaN
1,127690,SCI,SCI,NTL,,20.0,20,20,0,NaN,SCI,SCI,Grow,G,SCI,SCI,SCI,SCI
2,127692,Nat'l/Reg,Nat'l/Reg,NTL,,37616.0,16259,16259,8636,NaN,Extra Large,Partner,Retain,R,Nat'l/Reg Extra Large Partner,NXP,R,NaN
3,127695,Independent,Independent,,,20.0,20,20,0,NaN,Small,Unknown,Grow,G,Independent Small Unknown,ISU,U,U
4,127768,Strategic,Independent,,,240.0,120,120,0,NaN,Medium,Unknown,Grow,G,Independent Medium Unknown,IMU,U,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23746,913928,Independent,Independent,PAA,,482.0,179,179,97,Contract,Large,Partner,Retain,R,Independent Large Partner,ILP,R,NaN
23747,913931,Independent,Independent,GRO,PR,20.0,20,20,0,NaN,Small,Unknown,Grow,G,Independent Small Unknown,ISU,U,U
23748,913952,Independent,Independent,SPP,,20.0,20,20,0,NaN,Small,Unknown,Grow,G,Independent Small Unknown,ISU,U,U
23749,914022,Sm Emerging,Sm Emerging,GRO,SE1,20.0,20,20,0,NaN,Small,Unknown,Grow,G,Sm Emerging Small Unknown,SSU,U,U


In [125]:
seg_df2.dtypes

Customer Number                         object
Original                                object
Customer Spectrum                       object
CC23                                    object
CC28                                    object
Sum of Total Deaths - Parent           float64
Sum of Burials - Parent                  int64
Adj Burials                              int64
Burial Units Rolling 12 mo - Parent      int64
Buying Program                          object
Firm Size                               object
Partner / Vendor                        object
Retain / Grow                           object
Retain/Grow Code                        object
Bucket                                  object
CC16                                    object
CC17                                    object
CC17.1                                  object
dtype: object

In [126]:
updated = pd.merge(test, seg_df2, left_on='ship_to_id', right_on='Customer Number', how='left')

In [129]:
updated.head(10)

,ship_to_id,ship_to_name,parent_id,parent_name,city,state,country,division,region_code,region_name,territory_code,rep_name,open_status,customer_group_code,customer_group,customer_segment,Program Sub-Group 2,Program Sub-Group 1,customer_spectrum,rep_type,NAR Code,NAR Manager,std_industry_code,customer_type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,csc_id,csc_name,Serv Lvl,Delivery Frequency,Delivery Service Priority,buyer_type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,address_line_1,address_line_2,address_line_3,zip_code,lat,lon,territory_cd_short,territory_name,rep_first_name,rep_last_name,rep_nick_name,rep_name_alpha,territoryemail,regioncode,region_vp_full_name,region_vp_first_name,region_vp_last_name,region_vp_nick_name,region_name_2,rep_name_match,Customer Number,Original,Customer Spectrum,CC23,CC28,Sum of Total Deaths - Parent,Sum of Burials - Parent,Adj Burials,Burial Units Rolling 12 mo - Parent,Buying Program,Firm Size,Partner / Vendor,Retain / Grow,Retain/Grow Code,Bucket,CC16,CC17,CC17.1
0,131453,02550 CAPITAL MERCHANDISE & SERVICES,131453,02550 CAPITAL MERCHANDISE & SERVICES,WHEAT RIDGE,CO,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,CO,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,1682.0,1682,1682,1682,,8014,CSC DENVER INVENTORY,M T W R F,5 x per week,0,Non-Buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8578 W 32ND AVE,,,80033,39.7621,-105.092974,G1,HOUSE ACCOUNTS,G1 HOUSE,,G1 HOUSE,G1 HOUSE ACCOUNTS,,E1,,,,,HOUSE REGION,G1 HOUSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,152738,01227 RIVERSIDE MEMORIAL CHAPEL,152738,01227 RIVERSIDE MEMORIAL CHAPEL,NEW YORK,NY,US,HOUSE

In [130]:
updated.to_csv('../data/processed/new-seg-scm-edw-10-24-geocoded-cleaned.csv')